# GCC's ssc-cdi : tomography pipeline for Cateretê v0.1
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/
- #### To use a test sample:
    - dic["sinogram_path"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_complex/phantom_complex.hdf5"

## Imports

In [ ]:
%matplotlib widget
import numpy as np, matplotlib.pyplot as plt
from sscCdi import read_data, define_paths, tomo_sort, tomo_crop, tomo_unwrap, tomo_equalize, preview_angle_projection, tomo_alignment, tomo_recon, tomo_equalize3D, deploy_visualizer

## Load dictionary

In [ ]:
dic = {} # or load from template file

## Load data

In [ ]:
dic["recon_method"]  = "ptycho" # ptycho or pwcdi
dic["contrast_type"] = "magnitude" # phase or magnitude
dic["sinogram_path"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/phantom_complex/phantom_complex.hdf5"

dic = define_paths(dic)
obj, angles = read_data(dic)
print(f"Object shape = {obj.shape} \t Number of angles: {angles.shape}")

In [ ]:
deploy_visualizer(obj,title='Original sinogram')

## Sort data

In [ ]:
dic = tomo_sort(dic,obj, angles)

In [ ]:
deploy_visualizer(np.load(dic["ordered_object_filepath"]),title="Ordered sinogram")

## Crop

In [ ]:
dic["top_crop"]    = 20
dic["bottom_crop"] = 1
dic["left_crop"]   = 1
dic["right_crop"]  = 1
dic = tomo_crop(dic)

In [ ]:
deploy_visualizer(np.load(dic["cropped_sinogram_filepath"]),title="Cropped sinogram")

## Unwrap

In [ ]:
dic["bad_frames_before_unwrap"] = []
dic = tomo_unwrap(dic)

In [ ]:
deploy_visualizer(np.load(dic["unwrapped_sinogram_filepath"]),title="Unwrapped sinogram")

## 2D Equalization

In [ ]:
dic["bad_frames_before_equalization"] = []
dic["equalize_gradient"] = 0
dic["equalize_global_offset"] = False
dic["equalize_invert"] = False
dic["equalize_local_offset"] = [ 0, 20, 0, 20 ]
dic["equalize_outliers"] = 0

dic = tomo_equalize(dic)

In [ ]:
deploy_visualizer(np.load(dic["equalized_sinogram_filepath"]),title="Equalized sinogram")

## Alignment

In [ ]:
dic["wiggle_sinogram_selection"] = dic["equalized_sinogram_filepath"]
dic["step_percentage"] = 100 # need to project irregular angle steps to a regular grid?
preview_angle_projection(dic)

In [ ]:
dic["CPUs"] = 96
dic["bad_frames_before_wiggle"] = [] 
dic["wiggle_reference_frame"] = 10
dic = tomo_alignment(dic)

In [ ]:
deploy_visualizer(np.load(dic["wiggle_sinogram_filepath"]),title="Aligned sinogram",cmap='gray',axis=1)

## Tomography

In [ ]:
dic["GPUs"] = [0]
dic["tomo_algorithm"] = "EEM"
dic["tomo_iterations"] = 40
dic["tomo_regularization"] = False
dic["tomo_regularization_param"] = 0.001

dic = tomo_recon(dic)

In [ ]:
deploy_visualizer(np.load(dic["reconstruction_filepath"]),title="Recon",cmap='gray',axis=0)

## 3D Equalization

In [ ]:
dic["tomo_remove_outliers"] = 0
dic["tomo_threshold"] = 1000.0
dic["tomo_local_offset"] = []
dic["tomo_mask"] = []

dic = tomo_equalize3D(dic)

In [ ]:
deploy_visualizer(np.load(dic["eq_reconstruction_filepath"]),title="Equalized Recon",cmap='gray',axis=0)